## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from a) your home and b) work locations. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - id --> unique number (e.g. 0 and 1)
    - addr --> address of your work and home (you don't need to reveal your home / work if you don't want to, these can be whatever two addresses from Helsinki!)
    
Save the text file into this repository.
    
 - Read those addresses into Pandas and convert the addresses to Point objects using the geocoding functionalities of Geopandas
 - Find out the nearest shopping center to these points using the techniques shown during the lesson. You can use the shopping center addresses you geocoded in Problem 1 as the destination points.  

In [65]:
# Define filepath
fp = r"activity_locations.txt"

import pandas as pd
import geopandas as gpd
# Read the data (replace "None" with your own code)
data = pd.read_csv(fp,sep=";")
data

,id,addr
0,0,"Gustaf Hällströminkatu 2, 00560 Helsinki"
1,1,"Talvelantie 4, 00700 Helsinki"


In [66]:
from geopandas.tools import geocode

# Geocode addresses with Nominatim backend
geo = geocode(data['addr'], provider = 'nominatim', user_agent = 'autogis_student_xx')
geo.head()

,geometry,address
0,POINT (24.962130306481 60.20500895),"Tiedekirjasto, 2, Gustaf Hällströmin katu, Kum..."
1,POINT (25.0012582628003 60.25398535),"Malmin terveysasema ja hammashoitola, 4, Talve..."


In [67]:
## read the shopping center
# Read the data (replace "None" with your own code)
shop_data = pd.read_csv("shopping_centers.txt",sep=";")
# Geocode addresses with Nominatim backend
shop_geo = geocode(shop_data['addr'], provider = 'nominatim', user_agent = 'autogis_student_xx')

In [68]:
# check the crs of geo and shop_geo
if (geo.crs != shop_geo.crs):
    geo = geo.to_crs(shop_geo.crs)
print(geo.crs)

{'init': 'epsg:4326', 'no_defs': True}


In [69]:
# join the shop_geo and shop_data
shop_geo_data = shop_geo.join(shop_data)
shop_geo_data.head()

,geometry,address,id,name,addr
0,POINT (25.0829394427168 60.21170635),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1,Itis,"Itäkatu 1-7, 00930 Helsinki"
1,POINT (24.936116 60.171405),"Mannerheimintie, Keskusta, Kluuvi, Eteläinen s...",2,Forum,"Mannerheimintie 14–20, 00100 Helsinki"
2,POINT (24.73833665221 60.1612882),"Iso Omena, 11, Piispansilta, Espoo, Helsingin ...",3,Iso-omena,"Piispansilta 11, 02230 Espoo"
3,POINT (24.8099222238214 60.21867775),"3-9, Leppävaarankatu, Etelä-Leppävaara, Leppäv...",4,Sello,"Leppävaarankatu 3-9, 02600 Espoo"
4,POINT (25.0364526 60.3091871),"Vantaa, Helsingin seutukunta, Uusimaa, Etelä-S...",5,Jumbo,01510 Vantaa


In [70]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
# find the nearest point
destinations = MultiPoint(shop_geo_data["geometry"])

# find the nearest for wach point
nearest_orig1 = nearest_points(geo.loc[0,"geometry"], destinations)
nearest_orig2 = nearest_points(geo.loc[1,"geometry"], destinations)

In [71]:
shopname1 = shop_geo_data[ shop_geo_data["geometry"]==nearest_orig1[1]]
shopname2 = shop_geo_data[ shop_geo_data["geometry"]==nearest_orig2[1]]

In [73]:
print(shopname1["name"])
print(shopname2["name"])

5    REDI
Name: name, dtype: object
4    Jumbo
Name: name, dtype: object


In [75]:
print("The nearest shopping center to my workplace and home is {0} and {1},respectively."
      .format(shopname1.loc[5,"name"], shopname2.loc[4,"name"]))

The nearest shopping center to my workplace and home is REDI and Jumbo,respectively.


Can you think of other application cases for the nearest neighbour analysis?

Yes. To find the nearest bank, school, and hostiptal near home.

### problem
when i used the following address to do the geocode, there is a mistake "'NoneType' object is not iterable "

1;Raidepolku 4 Puistolan Juna-asema raide 3-4, 00760 Helsinki